In [1]:
install.packages("dplyr")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
install.packages("ggplot2")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [3]:
install.packages("usmap")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [54]:
install.packages("scales")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [4]:
library(usmap)

In [5]:
library(ggplot2)

In [6]:
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [7]:
#Extracted HCPCS code from Physician and Other Supplier Data CY 2017 base of LIST OF MEDICARE TELEHEALTH SERVICES in CY 2017, saved as MedicareAllTelehealth2017.csv
setwd ("/Users/ruyazhang/downloads")

In [49]:
dat1 <- read.csv("MedicareAllTelehealth2017.csv")

In [50]:
#medicare1 <- dat1[,c("State", "Gender", "Credentials", "City", "ProviderType")]
#summary(medicare1, maxsum = 100)

In [51]:
#aggregate the total NumberServices in 59 states
aggNumServices <- aggregate(NumberServices ~ State, dat1, sum)

In [52]:
aggNumServices[order(aggNumServices$NumberServices),]

,State,NumberServices
4,AP,589.0
1,AE,4460.0
58,XX,4492.0
29,MP,4804.0
59,ZZ,5811.0
14,GU,19324.0
52,VI,25649.0
2,AK,194642.0
57,WY,216437.5
44,PR,223525.0


In [53]:
#Summary statistic (mean, standard deviation, median, Interquartile Range(25th and 75th percentile), min,max)
#summary(aggNumServices)

In [54]:
#read dataset from CMS for medicare benificiaries(FFS) CY 2017
setwd ("/Users/ruyazhang/downloads")

In [55]:
dat2 <- read.csv("StateCountyFFS2017.csv")

In [56]:
# The data contains county level, extrated State Total FFS population, 53 states
stateBeneficiaries <- select(filter(dat2, County == "STATE TOTAL"), c(State, Beneficiaries) )

In [57]:
#print(stateBeneficiaries)

In [58]:
#df <- merge(x=stateBeneficiaries, y=aggNumServices, by = "State", all=TRUE, sort=TRUE)

In [59]:
#merge is a function in dplyr package, this merge reduced from 55 states in telehealth data to 53 states from FFS
df <- merge(x=stateBeneficiaries, y=aggNumServices, by = "State")

In [60]:
#Culculation NumberServices/Beneficiaries(FFS) and converted number to  numeric
df$ServicePerFFS <- df[,3]/as.numeric(as.character(df[,2]))

In [23]:
#print(df)

In [73]:
#write.csv(df, "stateAll.csv")

In [74]:
#setwd("/Users/ruyazhang/downloads")

In [75]:
#df <- read.csv("stateAll.csv")

In [76]:
#df <- df[,-1]

In [61]:
#rename the 4 columns in below format, in order to merge to Usmap dataset
names(df) <- c("abbr","FFSpop","Service","ServicePerFFS")

In [62]:
#merge to Usmap dataset by "abbr"
df <- merge(statepop, df, by="abbr")

In [79]:
#df$ServicePerFFS <- df$ServicePerFFS *100

In [95]:
#print(df)

In [26]:
#plot_usmap(data = df, values = "ServicePerFFS", color = "red") + 
#  scale_fill_continuous(name = "ServicePerFFS1k (2017)", 
#                        type = "viridis", 
#                        labels = paste(round(quantile(df$ServicePerFFS),2),
#                                       names(quantile(df$ServicePerFFS)), "")) + 
#  theme(legend.position = "right") 

In [82]:
#setwd ("/Users/ruyazhang/downloads")

In [83]:
#dat1 <- read.csv("MedicareAllTelehealth2017.csv")

In [63]:
#aggregate unique NPI from MedicareAllTelehealth2017.csv dataset for total MD in 55 states
UniqueNPI <- summarize(group_by(dat1, State), unique_NPI = n_distinct(NPI))

`summarise()` ungrouping output (override with `.groups` argument)


In [64]:
#Rename to below
names(UniqueNPI) <- c("abbr","NumberOfProvider")

In [65]:
#merge total MD(providers) to df
df <- merge(df, UniqueNPI, by="abbr")

In [30]:
#plot_usmap(data = df, values = "NumberOfProvider", color = "red") + 
#  scale_fill_continuous(name = "NumberOfProvider(2017)", type = "viridis", label = scales::comma) + 
#  theme(legend.position = "right")

In [66]:
#Culculation Provider/Beneficiaries(FFS) and converted number to numeric
df$ProviderPerFFS <- df[,8]/as.numeric(as.character(df[,5]))
#df$ProviderPerFFS <- df$ProviderPerFFS *10000

In [33]:
#plot_usmap(data = df, values = "ProvidersPerFFS", color = "red") + 
#  scale_fill_continuous(name = "ProvidersPerFFS10k(2017)", type = "viridis", label = scales::comma) + 
#  theme(legend.position = "right")

In [67]:
#Added new column for the year of the datasets
df$Year <- 2017

In [68]:
#Extracted HCPCS code from Physician and Other Supplier Data CY 2016 base of LIST OF MEDICARE TELEHEALTH SERVICES in CY 2016, saved as MedicareAllTelehealth2016.csv
setwd ("/Users/ruyazhang/downloads")
dat3 <- read.csv("MedicareAllTelehealth2016.csv")

In [69]:
#aggregate the total NumberServices in 60 states
aggNumServices <- aggregate(NumberServices ~ State, dat3, sum)
aggNumServices[order(aggNumServices$NumberServices),]

,State,NumberServices
1,AA,156.0
5,AP,1204.0
59,XX,2119.0
2,AE,2514.0
30,MP,3134.0
60,ZZ,6363.0
53,VI,22931.0
15,GU,24441.0
3,AK,192151.0
58,WY,223518.0


In [70]:
#read dataset from CMS for medicare benificiaries(FFS) CY 2016
setwd ("/Users/ruyazhang/downloads")
dat4 <- read.csv("StateCountyFFS2016.csv")

In [71]:
# The data contains county level, extrated State Total FFS population, 53 states
stateBeneficiaries <- select(filter(dat4, County == "STATE TOTAL"), c(State, Beneficiaries) )

In [72]:
#merge is a function in dplyr package, this merge reduced from 60 states in telehealth data to 53 states from FFS
df2 <- merge(x=stateBeneficiaries, y=aggNumServices, by = "State")

In [73]:
#Culculation NumberServices/Beneficiaries(FFS) and converted number to numeric
df2$ServicePerFFS <- df2[,3]/as.numeric(as.character(df2[,2]))

In [74]:
#rename the 4 columns in below format, in order to merge to Usmap dataset, and merge to Usmap dataset by "abbr"
names(df2) <- c("abbr","FFSpop","Service","ServicePerFFS")
df2 <- merge(statepop, df2, by="abbr")

In [75]:
#aggregate unique NPI from MedicareAllTelehealth2017.csv dataset for total MD in 55 states
UniqueNPI <- summarize(group_by(dat3, State), unique_NPI = n_distinct(NPI))
#Rename to below
names(UniqueNPI) <- c("abbr","NumberOfProvider")
#merge total MD(providers) to df2
df2 <- merge(df2, UniqueNPI, by="abbr")

`summarise()` ungrouping output (override with `.groups` argument)


In [76]:
#Culculation Providers/Beneficiaries(FFS) and converted number to numeric
df2$ProviderPerFFS <- df2[,8]/as.numeric(as.character(df2[,5]))

In [77]:
#Added new column for the year of the datasets
df2$Year <- 2016

In [78]:
#Stack two dataframe df and df2
dfAll <- rbind(df, df2)